In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.node_parser import SimpleFileNodeParser
from llama_index.readers.file import FlatReader
from llama_parse import LlamaParse

from pathlib import Path
import os
from dotenv import load_dotenv
import nest_asyncio

load_dotenv()
nest_asyncio.apply()

In [ ]:
os.environ["LLAMA_CLOUD_API_KEY"] = os.getenv("LLAMA_CLOUD_API_KEY")

In [ ]:
file_dir = "../data"

docs = SimpleDirectoryReader(
    input_dir=file_dir,
    recursive=True,
).load_data()

In [ ]:
for i in docs[0]:
    print(i)

In [ ]:
docs = SimpleDirectoryReader(input_dir=file_dir, filename_as_id=True).load_data()

for i in docs[0]:
    print(i)

In [ ]:
print(docs[0])

In [ ]:
reader = SimpleWebPageReader(html_to_text=True)
pages = reader.load_data(
    urls=["https://www.sciencedaily.com/releases/2025/05/250527124428.htm"]
)

print(pages[0].text)

In [ ]:
# LLamaParse
parser = LlamaParse(result_type="text")
file_extractor = {".pdf": parser}

reader = SimpleDirectoryReader(
    input_dir=file_dir,
    file_extractor=file_extractor,
)

docs = reader.load_data()

In [ ]:
print(docs[0])

In [ ]:
md_docs = FlatReader().load_data(Path("../docs/biography/toan.md"))
parser = SimpleFileNodeParser()
md_nodes = parser.get_nodes_from_documents(md_docs, show_progress=True)
print(md_nodes[0])